<a href="https://colab.research.google.com/github/Fuenfgeld/DMA2023TeamB/blob/main/ETL_Prozess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Datawarehouse creation and transformation process for data from COVID-19 source database to datawarehouse db



# Importing required libraries and mountig googledrive with scharedfolder location containig source database

In [1]:
import sqlite3
from sqlite3 import Error

In [2]:
# mount drive to access database
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# Defining input path for Data form source database and  and output data path for Datawarehouse 


In [5]:
# type of patients
patient_type = "covid19"

In [6]:
# path of source database in shared googledrive Folder
DB_SOURCE_PATH = f"/content/drive/Shareddrives/DMA_2023_D/source_dbs/source_covid19_test.db"

'/content/drive/MyDrive/Gruppe_B/source_covid19_test.db'

In [7]:
# path of datawarehouse with Filename DWH_COVID-19_2023.db
DB_DWH_PATH = f"/content/drive/Shareddrives/DMA_2023_D/source_dbs/DWH_COVID-19_2023.db"

# Creating Datawarehouse 

In [8]:
class DB(object):
  def __init__(self, db_file):
    self.conn = sqlite3.connect(db_file)
    self.cur = self.conn.cursor()
    self.__init_db()
  
  def __del__(self):
      self.conn.commit()
      self.conn.close()

  def __init_db(self):
    # sql queries to create tables in Datawarehouse 

    #  sql query to create patients_info table -> Isoliert ID, Geburts-/Todesdatum, Gender
    create_patients_info = """CREATE TABLE IF NOT EXISTS patients_info (
                           Id STRING PRIMARY KEY,
                           BIRTHDATE DATE,
                           DEATHDATE DATE,
                           GENDER STRING,
                       );"""

    #  sql query to create disease table -> Disease Table nicht benötigt


    # sql query to create condition table -> Encounter ex, Encounters Table nicht gebraucht, Description as Code
    create_conditions_info = """CREATE TABLE IF NOT EXISTS conditions_info (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           CODE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_info (Id) 
                       );"""

    # sql query to create careplans table -> nicht benötigt
  

    # sql query to create procedures table -> encounter ex, Reduktion auf Pat.ID und SNOMED Code
    create_procedures_info = """CREATE TABLE IF NOT EXISTS procedures_info (
                           DATE DATE,
                           PATIENT STRING,
                           CODE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_info (Id) 
                       );"""

    # sql query to create medications_info table -> DESCRIPTION BELASSEN, DA MULTIPLE Medikamente
    create_medications_info = """CREATE TABLE IF NOT EXISTS medications_info (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           CODE STRING,
                           DESCRIPTION STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_info (Id)   
                       );"""

    # sql query to create observations table -> Table nicht benötigt


    # sql query to create devices table
    create_devices_info = """CREATE TABLE IF NOT EXISTS devices_info (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           CODE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_info (Id)                 
                       );"""
   
   # sql query to create Imaging_studies table -> ist leer
   # zusätzlich sollte ein immunizations table angelegt werden:
   # sql query to create immunizations table
    create_immunizations_info = """CREATE TABLE IF NOT EXISTS devices_info (
                           START DATE,
                           STOP DATE,
                           PATIENT STRING,
                           CODE STRING,
                           FOREIGN KEY (PATIENT)
                              REFERENCES patients_info (Id)                 
                       );"""


    create_tables = [create_patients_info, # patient data
                     create_conditions_info, create_procedures_info, create_devices_info,  # symptoms and procedure data
                     create_medications_info, # medication data
                     create_immunizations_info # immunization info
                     ]
     

    if self.conn is not None:
      # self.cur.execute(f"drop table if exists medications_info")
      for query in create_tables:
          self.cur.execute(query)
    else:
      print('Connection to database failed')



#ETL/ELT (Extract, transform, load )

In [9]:
# exporting queries
class SqlQuery:
  def __init__(self, source_table, column_names, sink_table):
    self.source_table = source_table
    self.column_numbers = len(column_names)
    self.column_names = ', '.join(column_names)
    self.sink_table = sink_table

  def extract_query(self):
    return 'SELECT ' + self.column_names + ' FROM ' + self.source_table 

  def load_query(self):
    values_str = '?,' * self.column_numbers
    # print("*****", values_str, column_names, column_numbers)
    values_str = values_str[:-1]
    return 'INSERT OR REPLACE INTO ' + self.sink_table + ' VALUES (' + values_str + ')'

    # return 'INSERT INTO ' + self.sink_table + '(' + self.column_names + ') VALUES (' + values_str + ')'
  



In [10]:
def etl(query, source_cnx, target_cnx):
  # extract data from source db
  source_cursor = source_cnx.cursor()
  source_cursor.execute(query.extract_query())
  data = source_cursor.fetchall()
  source_cursor.close()

  # load data into warehouse db
  if data:
    target_cursor = target_cnx.cursor()
    target_cursor.executemany(query.load_query(), data)
    print('data loaded to warehouse db') 
    target_cnx.commit()
    target_cursor.close()
  else:
    print('data is empty')


def etl_process(queries, target_cnx, db_source):
  """
  queries: list
        a list of queries
  target_cnx: SQLite connection
  db_source: str
        path of source database      
  
  """  
  # establish source db connection
  try:
    source_cnx = sqlite3.connect(db_source)
  except Error as err:
    print(err)
  
  # loop through sql queries
  for query in etl_queue:
    etl(query, source_cnx, target_cnx)
    
  # close the source db connection
  source_cnx.close()

In [11]:
# create Datawarehouse
dwh_db = DB(DB_DWH_PATH)

In [12]:
print('starting etl')   
# list for iteration
etl_queue = []

# patient table
patients_columns = ['Id', 'BIRTHDATE', 'DEATHDATE', 'GENDER']
sql_query_patients = SqlQuery("patients", patients_columns, "patients_info")
etl_queue.append(sql_query_patients)

# symptoms and procedures table
conditions_columns = ['START', 'STOP', 'PATIENT', 'CODE']
sql_query_conditions = SqlQuery("conditions", conditions_columns, "conditions_info")
etl_queue.append(sql_query_conditions)

procedures_columns = ['DATE', 'PATIENT', 'CODE']
sql_query_procedures = SqlQuery("procedures", procedures_columns, "procedures_info")
etl_queue.append(sql_query_procedures)

devices_columns = ['START', 'STOP', 'PATIENT', 'CODE']
sql_query_devices = SqlQuery("devices", devices_columns, "devices_info")
etl_queue.append(sql_query_devices)

# medications table
medications_columns = ['START', 'STOP', 'PATIENT', 'CODE', 'DESCRIPTION']
sql_query_medications = SqlQuery("medications", medications_columns, "medications_info")
etl_queue.append(sql_query_medications)

# immunizations table
immunizations_columns = ['START', 'STOP', 'PATIENT', 'CODE']
sql_query_careplans = SqlQuery("careplans", careplans_columns, "careplans_info")
etl_queue.append(sql_query_careplans)

# list for iteration
# etl_queue

starting etl


In [13]:
# establish connection for target database (sql-server)
target_cnx = dwh_db.conn
etl_process(etl_queue, target_cnx, DB_SOURCE_PATH)

data loaded to warehouse db
data is empty
data loaded to warehouse db
data loaded to warehouse db
data loaded to warehouse db
data loaded to warehouse db
data loaded to warehouse db
data loaded to warehouse db
data is empty


In [14]:
target_cnx.commit()

In [ ]:
# target_cnx.close()

In [15]:
# # check list of tables
dwh_cursor = target_cnx.cursor()
# dwh_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# print(dwh_cursor.fetchall())


In [16]:
# # check columns in a table
dwh_cursor.execute('PRAGMA table_info(' + "patients_info" + ');')
dwh_cursor.fetchall()

[(0, 'Id', 'STRING', 0, None, 1),
 (1, 'BIRTHDATE', 'DATE', 0, None, 0),
 (2, 'DEATHDATE', 'DATE', 0, None, 0),
 (3, 'SSN', 'STRING', 0, None, 0),
 (4, 'DRIVERS', 'STRING', 0, None, 0),
 (5, 'PASSPORT', 'STRING', 0, None, 0),
 (6, 'PREFIX', 'STRING', 0, None, 0),
 (7, 'FIRST', 'STRING', 0, None, 0),
 (8, 'LAST', 'STRING', 0, None, 0),
 (9, 'SUFFIX', 'STRING', 0, None, 0),
 (10, 'MAIDEN', 'STRING', 0, None, 0),
 (11, 'MARITAL', 'STRING', 0, None, 0),
 (12, 'RACE', 'STRING', 0, None, 0),
 (13, 'ETHNICITY', 'STRING', 0, None, 0),
 (14, 'GENDER', 'STRING', 0, None, 0),
 (15, 'BIRTHPLACE', 'STRING', 0, None, 0),
 (16, 'ADDRESS', 'STRING', 0, None, 0),
 (17, 'CITY', 'STRING', 0, None, 0),
 (18, 'STATE', 'STRING', 0, None, 0),
 (19, 'COUNTRY', 'STRING', 0, None, 0),
 (20, 'ZIP', 'STRING', 0, None, 0),
 (21, 'LAT', 'INTEGER', 0, None, 0),
 (22, 'LON', 'INTEGER', 0, None, 0),
 (23, 'HEALTHCARE_EXPENSES', 'INTEGER', 0, None, 0),
 (24, 'HEALTHCARE_COVERAGE', 'INTEGER', 0, None, 0)]

In [17]:
dwh_cursor.execute("SELECT Id, BIRTHDATE, GENDER from patients_info")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

('2f114e21-8b95-e9eb-2e9e-4b950b059cfa', '2002-05-05', 'Terisa250', 'Kreiger457', '', 'F')
('2812f4fc-6aba-0033-728b-7e569bb8adcc', '1942-05-24', 'Romeo514', 'Morar593', 'M', 'M')
('9b1a203a-6368-49a5-0c79-45c32729bc17', '2013-06-15', 'Jamaal34', 'Hessel84', '', 'M')
('52a250a9-24e4-ec9b-91d6-c38cf1b10fc0', '1997-11-16', 'Kenya524', 'Leuschke194', '', 'F')
('ad5525c0-fdab-94dc-018f-d7209c72bdef', '1965-08-28', 'Clark193', 'Little434', 'M', 'M')
('7f30313f-98e9-2cc2-68a5-6f9973f9ad5a', '1974-12-06', 'Simona661', 'Green467', 'M', 'F')
('f7dddcba-380e-7fd4-20b0-da086ce5dc3a', '1991-08-19', 'Moira454', 'Quitzon246', 'M', 'F')
('21e040b3-3c29-6042-1dba-da47dee67c18', '1952-12-25', 'Guillermo498', 'Cepeda588', 'M', 'M')
('b9f7feb0-45ac-55ed-4ee2-be12ad599416', '1980-02-24', 'Geralyn932', 'Steuber698', 'S', 'F')
('2b2d8855-c163-3838-a719-995a3cefca33', '1992-10-19', 'Gracia333', 'Lockman863', 'M', 'F')


In [18]:
dwh_cursor.execute("SELECT PATIENT, CODE from procedures_info")
rows = dwh_cursor.fetchall()
for row in rows[:10]:
  print(row)

('52a250a9-24e4-ec9b-91d6-c38cf1b10fc0', 261352009, 'Face mask (physical object)', 4.67)
('2f114e21-8b95-e9eb-2e9e-4b950b059cfa', 261352009, 'Face mask (physical object)', 3.1)
('2812f4fc-6aba-0033-728b-7e569bb8adcc', 261352009, 'Face mask (physical object)', 1.33)
('7f30313f-98e9-2cc2-68a5-6f9973f9ad5a', 261352009, 'Face mask (physical object)', 3.56)
('9b1a203a-6368-49a5-0c79-45c32729bc17', 261352009, 'Face mask (physical object)', 1.95)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 261352009, 'Face mask (physical object)', 2.59)
('2812f4fc-6aba-0033-728b-7e569bb8adcc', 180325003, 'Electrical cardioversion', 44895.87)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 399208008, 'Plain chest X-ray (procedure)', 6211.16)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 371908008, 'Oxygen administration by mask (procedure)', 516.65)
('ad5525c0-fdab-94dc-018f-d7209c72bdef', 431182000, 'Placing subject in prone position (procedure)', 516.65)
